# EDA + Cleaning


In [1]:
import pandas as pd
import numpy as np
import ast
import prince 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest


In [2]:
df = pd.read_json(
    "C:/Users/memer/OneDrive/Documentos/Eme/UNIVERSIDAD/2025-1/ETL/Tercer Cohorte/Proy85/proy85/data/MLA_100k.jsonlines", 
    lines=True
)

In [3]:
df.shape
df.head(10)

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,...,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,geolocation,sold_quantity,available_quantity
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",None,[],new,None,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",74952096,...,active,None,NaN,NaN,1,2015-09-05 20:42:53+00:00,http://articulo.mercadolibre.com.ar/MLA-578052...,"{'latitude': -34.6280698, 'longitude': -58.398...",0,1
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,None,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",42093335,...,active,None,NaN,NaN,1,2015-09-26 18:08:30+00:00,http://articulo.mercadolibre.com.ar/MLA-581565...,"{'latitude': -34.5935524, 'longitude': -58.505...",0,1
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",None,[],used,None,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",133384258,...,active,None,NaN,NaN,1,2015-09-09 23:57:07+00:00,http://articulo.mercadolibre.com.ar/MLA-578780...,"{'latitude': -34.6233907, 'longitude': -58.414...",0,1
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",None,[],new,None,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",143001605,...,active,None,NaN,NaN,1,2015-09-28 18:47:56+00:00,http://articulo.mercadolibre.com.ar/MLA-581877...,"{'latitude': -34.6281894, 'longitude': -58.492...",0,1
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",MI REPUTACION.,[],used,None,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",96873449,...,active,None,NaN,NaN,1,2015-08-24 22:07:20+00:00,http://articulo.mercadolibre.com.ar/MLA-576112...,"{'latitude': -34.6346547, 'longitude': -58.549...",0,1
5,"{'comment': '', 'longitude': -58.5373541, 'id'...",Sí,[],new,None,[],310.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",93122911,...,active,None,NaN,NaN,1,2015-08-30 14:24:01+00:00,http://articulo.mercadolibre.com.ar/MLA-576999...,"{'latitude': -34.5173889, 'longitude': -58.537...",0,1
6,"{'comment': '', 'longitude': -58.3767787, 'id'...",None,[],used,None,[],180.0,"{'local_pick_up': True, 'methods': [], 'tags':...",[],82247584,...,active,None,NaN,NaN,1,2015-10-06 22:32:41+00:00,http://articulo.mercadolibre.com.ar/MLA-583349...,"{'latitude': -34.6428264, 'longitude': -58.376...",0,1
7,"{'comment': '', 'longitude': -58.4927597, 'id'...",None,[],used,None,[],150.0,"{'local_pick_up': False, 'methods': [], 'tags'...","[{'description': 'Transferencia bancaria', 'id...",40903230,...,active,None,NaN,NaN,1,2015-10-14 19:36:52+00:00,http://articulo.mercadolibre.com.ar/MLA-584657...,"{'latitude': -34.6507416, 'longitude': -58.492...",0,1
8,"{'comment': '', 'longitude': -58.47917, 'id': ...",1 Ano,[],new,None,[],2352.0,"{'local_pick_up': True, 'free_methods': [{'rul...","[{'description': 'Tarjeta de crédito', 'id': '...",166958416,...,paused,QQNfOicE_o8,NaN,NaN,10,2015-09-30 21:50:22+00:00,http://articulo.mercadolibre.com.ar/MLA-582290...,"{'latitude': -34.58502, 'longitude': -58.47917}",0,10
9,"{'comment': '', 'longitude': -58.4900327, 'id'...",None,[],new,None,[],120.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Efectivo', 'id': 'MLAMO', 't...",20834268,...,active,None,NaN,NaN,2,2015-10-03 23:11:29+00:00,http://articulo.mercadolibre.com.ar/MLA-582819...,"{'latitude': -34.8113156, 'longitude': -58.490...",1,1


In [4]:
shipping_df = pd.json_normalize(df['shipping'])
shipping_df.columns = ['shipping_' + col for col in shipping_df.columns]
df = pd.concat([df.drop(columns='shipping'), shipping_df], axis=1)

In [5]:
geo_df = pd.json_normalize(df['geolocation'])
geo_df.columns = ['geo_' + col for col in geo_df.columns]
df = pd.concat([df.drop(columns='geolocation'), geo_df], axis=1)


In [6]:
methods = df['non_mercado_pago_payment_methods'].explode()
methods = methods.dropna().apply(lambda x: x['id'] if isinstance(x, dict) else None)
unique_methods = methods.dropna().unique()
for method in unique_methods:
    df[f'payment_{method}'] = df['non_mercado_pago_payment_methods'].apply(
        lambda x: int(any(m.get('id') == method for m in x)) if isinstance(x, list) else 0
    )
df.drop(columns='non_mercado_pago_payment_methods', inplace=True)


In [7]:
df.head(10)

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,seller_id,variations,location,...,payment_MLAMC,payment_MLAMS,payment_MLAVE,payment_MLACD,payment_MLAVS,payment_MLADC,payment_MLAAM,payment_MLAWT,payment_MLAMP,payment_MLABC
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",None,[],new,None,[],80.0,74952096,[],{},...,0,0,0,0,0,0,0,0,0,0
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,None,[],2650.0,42093335,[],{},...,0,0,0,0,0,0,0,0,0,0
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",None,[],used,None,[],60.0,133384258,[],{},...,0,0,0,0,0,0,0,0,0,0
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",None,[],new,None,[],580.0,143001605,[],{},...,0,0,0,0,0,0,0,0,0,0
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",MI REPUTACION.,[],used,None,[],30.0,96873449,[],{},...,0,0,0,0,0,0,0,0,0,0
5,"{'comment': '', 'longitude': -58.5373541, 'id'...",Sí,[],new,None,[],310.0,93122911,[],{},...,0,0,0,0,0,0,0,0,0,0
6,"{'comment': '', 'longitude': -58.3767787, 'id'...",None,[],used,None,[],180.0,82247584,[{'attribute_combinations': [{'value_id': '920...,{},...,0,0,0,0,0,0,0,0,0,0
7,"{'comment': '', 'longitude': -58.4927597, 'id'...",None,[],used,None,[],150.0,40903230,[],{},...,0,0,0,0,0,0,0,0,0,0
8,"{'comment': '', 'longitude': -58.47917, 'id': ...",1 Ano,[],new,None,[],2352.0,166958416,[],{},...,0,0,0,0,0,0,0,0,0,0
9,"{'comment': '', 'longitude': -58.4900327, 'id'...",None,[],new,None,[],120.0,20834268,[{'attribute_combinations': [{'value_id': '920...,{},...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.isnull().sum()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.isnull().sum())

seller_address                      0
warranty                        60896
sub_status                          0
condition                           0
seller_contact                  97781
deal_ids                            0
base_price                          0
seller_id                           0
variations                          0
location                            0
site_id                             0
listing_type_id                     0
price                               0
attributes                          0
buying_mode                         0
tags                                0
listing_source                      0
parent_item_id                  23011
coverage_areas                      0
category_id                         0
descriptions                        0
last_updated                        0
international_delivery_mode         0
pictures                            0
id                                  0
official_store_id               99182
differential

In [9]:
df = df.explode('deal_ids')
df = df.explode('sub_status')
df = df.explode('variations')

nan_deal_ids = df['deal_ids'].isna().sum()
nan_sub_status = df['sub_status'].isna().sum()
nan_variations = df['variations'].isna().sum()

print(f"NaN en 'deal_ids': {nan_deal_ids}")
print(f"NaN en 'sub_status': {nan_sub_status}")
print(f"NaN en 'variations': {nan_variations}")


NaN en 'deal_ids': 109625
NaN en 'sub_status': 109765
NaN en 'variations': 91822


In [10]:
cols_to_drop = list(set([
    'id',
    'warranty',
    'seller_id',
    'currency_id',
    'base_price',
    'shipping_local_pick_up',
    'picture_width',
    'sub_status',
    'location',
    'attributes',
    'tags',
    'seller_contact',
    'differential_pricing',
    'original_price',
    'subtitle',
    'shipping_dimensions',
    'deal_ids',
    'sub_statys',
    'variations',
    'official_store_id',
    'catalog_product_id',
    'parent_item_id',
    'video_id',
    'shipping_tags',
    'seller_address',
    'listing_source',
    'coverage_areas',
    'descriptions',
]))


df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)



df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110867 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   condition                    110867 non-null  object             
 1   site_id                      110867 non-null  object             
 2   listing_type_id              110867 non-null  object             
 3   price                        110867 non-null  float64            
 4   buying_mode                  110867 non-null  object             
 5   category_id                  110867 non-null  object             
 6   last_updated                 110867 non-null  object             
 7   international_delivery_mode  110867 non-null  object             
 8   pictures                     110867 non-null  object             
 9   accepts_mercadopago          110867 non-null  bool               
 10  thumbnail                    110867 no

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 110867 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   condition                    110867 non-null  object             
 1   site_id                      110867 non-null  object             
 2   listing_type_id              110867 non-null  object             
 3   price                        110867 non-null  float64            
 4   buying_mode                  110867 non-null  object             
 5   category_id                  110867 non-null  object             
 6   last_updated                 110867 non-null  object             
 7   international_delivery_mode  110867 non-null  object             
 8   pictures                     110867 non-null  object             
 9   accepts_mercadopago          110867 non-null  bool               
 10  thumbnail                    110867 no

In [12]:
df['picture_size'] = df['pictures'].apply(
    lambda lst: [d.get('size') for d in lst] if isinstance(lst, list) else None
)


In [13]:
columnas_fecha = ['last_updated', 'date_created', 'stop_time', 'start_time'] 

for col in columnas_fecha:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    df[col + '_year'] = df[col].dt.year
    df[col + '_month'] = df[col].dt.month
    df.drop(col, axis=1, inplace=True)


In [14]:
df = df.loc[:, ~df.columns.duplicated()]
for col in df.columns:
    try:
        print(f"{col}: {df[col].nunique()}")
    except TypeError:
        print(f"{col}: contiene listas o dicts no contables")




condition: 2
site_id: 1
listing_type_id: 7
price: 10331
buying_mode: 3
category_id: 10907
international_delivery_mode: 1
pictures: contiene listas o dicts no contables
accepts_mercadopago: 2
thumbnail: 97968
title: 98823
automatic_relist: 2
secure_thumbnail: 97968
status: 4
initial_quantity: 450
permalink: 100000
sold_quantity: 317
available_quantity: 468
shipping_methods: contiene listas o dicts no contables
shipping_free_shipping: 2
shipping_mode: 4
shipping_free_methods: contiene listas o dicts no contables
geo_latitude: 26116
geo_longitude: 26080
payment_MLATB: 2
payment_MLAWC: 2
payment_MLAMO: 2
payment_MLAOT: 2
payment_MLAMC: 2
payment_MLAMS: 2
payment_MLAVE: 2
payment_MLACD: 2
payment_MLAVS: 2
payment_MLADC: 2
payment_MLAAM: 2
payment_MLAWT: 2
payment_MLAMP: 2
payment_MLABC: 2
picture_size: contiene listas o dicts no contables
last_updated_year: 2
last_updated_month: 12
date_created_year: 3
date_created_month: 12
stop_time_year: 3
stop_time_month: 12
start_time_year: 3
start_tim

In [15]:
df.drop(columns=['shipping_methods', 'shipping_free_methods'], inplace=True)

In [16]:
df['picture_size'] = df['picture_size'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['first_picture_size'] = df['picture_size'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
df['picture_width'] = df['first_picture_size'].apply(lambda x: x.split('x')[0] if isinstance(x, str) and 'x' in x else np.nan)
df['picture_height'] = df['first_picture_size'].apply(lambda x: x.split('x')[1] if isinstance(x, str) and 'x' in x else np.nan)
df[['picture_width', 'picture_height']] = df[['picture_width', 'picture_height']].astype(float)


In [17]:
df.head(10)

,condition,site_id,listing_type_id,price,buying_mode,category_id,international_delivery_mode,pictures,accepts_mercadopago,thumbnail,...,last_updated_month,date_created_year,date_created_month,stop_time_year,stop_time_month,start_time_year,start_time_month,first_picture_size,picture_width,picture_height
0,new,MLA,bronze,80.0,buy_it_now,MLA126406,none,"[{'size': '500x375', 'secure_url': 'https://a2...",True,http://mla-s1-p.mlstatic.com/5386-MLA435206787...,...,9,2015,9,2015,11,2015,9,500x375,500.0,375.0
1,used,MLA,silver,2650.0,buy_it_now,MLA10267,none,"[{'size': '499x334', 'secure_url': 'https://a2...",True,http://mla-s1-p.mlstatic.com/23223-MLA20245018...,...,9,2015,9,2015,11,2015,9,499x334,499.0,334.0
2,used,MLA,bronze,60.0,buy_it_now,MLA1227,none,"[{'size': '375x500', 'secure_url': 'https://a2...",True,http://mla-s1-p.mlstatic.com/22076-MLA20223367...,...,9,2015,9,2015,11,2015,9,375x500,375.0,500.0
3,new,MLA,silver,580.0,buy_it_now,MLA86345,none,"[{'size': '441x423', 'secure_url': 'https://a2...",True,http://mla-s2-p.mlstatic.com/183901-MLA2043288...,...,10,2015,9,2015,12,2015,9,441x423,441.0,423.0
4,used,MLA,bronze,30.0,buy_it_now,MLA41287,none,"[{'size': '375x500', 'secure_url': 'https://a2...",True,http://mla-s2-p.mlstatic.com/13595-MLA13041807...,...,8,2015,8,2015,10,2015,8,375x500,375.0,500.0
5,new,MLA,bronze,310.0,buy_it_now,MLA1429,none,"[{'size': '250x250', 'secure_url': 'https://a2...",True,http://mla-s2-p.mlstatic.com/6207-MLA87256964_...,...,8,2015,8,2015,10,2015,8,250x250,250.0,250.0
6,used,MLA,free,180.0,buy_it_now,MLA352650,none,"[{'size': '500x375', 'secure_url': 'https://a2...",True,http://mla-s1-p.mlstatic.com/472901-MLA2044293...,...,10,2015,10,2015,12,2015,10,500x375,500.0,375.0
7,used,MLA,free,150.0,buy_it_now,MLA1227,none,"[{'size': '500x375', 'secure_url': 'https://a2...",True,http://mla-s2-p.mlstatic.com/9874-MLA200219365...,...,10,2015,10,2015,12,2015,10,500x375,500.0,375.0
8,new,MLA,gold_special,2352.0,buy_it_now,MLA352293,none,"[{'size': '500x500', 'secure_url': 'https://a2...",True,http://mla-s1-p.mlstatic.com/797901-MLA2043624...,...,10,2015,9,2015,11,2015,9,500x500,500.0,500.0
9,new,MLA,silver,120.0,buy_it_now,MLA121665,none,"[{'size': '375x500', 'secure_url': 'https://a2...",True,http://mla-s1-p.mlstatic.com/22802-MLA20237759...,...,10,2015,10,2015,12,2015,10,375x500,375.0,500.0


In [18]:
df.drop(['picture_size', 'first_picture_size', 'site_id', 'category_id', 'thumbnail', 'buying_mode'], axis=1, inplace=True)

In [19]:
df['automatic_relist'] = df['automatic_relist'].astype(int)
df.drop(columns=['pictures'], inplace=True)



In [20]:
cols_to_drop = df.columns[df.nunique() == 1]
df.drop(columns=cols_to_drop, inplace=True)


In [21]:
df['title_length'] = df['title'].astype(str).apply(len)
df.drop('title', axis=1, inplace=True)

In [23]:
df.drop('secure_thumbnail', axis=1, inplace=True)

In [24]:

df['last_updated_month_year'] = df['last_updated_year'].astype(str) + '-' + df['last_updated_month'].astype(str).str.zfill(2)
df['date_created_month_year'] = df['date_created_year'].astype(str) + '-' + df['date_created_month'].astype(str).str.zfill(2)
df['stop_time_month_year'] = df['stop_time_year'].astype(str) + '-' + df['stop_time_month'].astype(str).str.zfill(2)
df['start_time_month_year'] = df['start_time_year'].astype(str) + '-' + df['start_time_month'].astype(str).str.zfill(2)

df.drop(['last_updated_year', 'last_updated_month', 
         'date_created_year', 'date_created_month', 
         'stop_time_year', 'stop_time_month', 
         'start_time_year', 'start_time_month'], axis=1, inplace=True)


In [25]:
df.isnull().sum()

condition                    0
listing_type_id              0
price                        0
accepts_mercadopago          0
automatic_relist             0
status                       0
initial_quantity             0
permalink                    0
sold_quantity                0
available_quantity           0
shipping_free_shipping       0
shipping_mode                0
geo_latitude                 0
geo_longitude                0
payment_MLATB                0
payment_MLAWC                0
payment_MLAMO                0
payment_MLAOT                0
payment_MLAMC                0
payment_MLAMS                0
payment_MLAVE                0
payment_MLACD                0
payment_MLAVS                0
payment_MLADC                0
payment_MLAAM                0
payment_MLAWT                0
payment_MLAMP                0
payment_MLABC                0
picture_width              789
picture_height             789
title_length                 0
last_updated_month_year      0
date_cre

In [26]:
df = df.dropna()


In [27]:
df['condition'] = (df['condition'] == 'new').astype(int)

C:\Users\memer\AppData\Local\Temp\ipykernel_16472\437451288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['condition'] = (df['condition'] == 'new').astype(int)


In [28]:
numeric_df = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32'])
correlation_matrix = numeric_df.corr()

In [29]:
correlation_matrix.style.background_gradient(cmap='coolwarm', axis=None)

,condition,price,automatic_relist,initial_quantity,sold_quantity,available_quantity,payment_MLATB,payment_MLAWC,payment_MLAMO,payment_MLAOT,payment_MLAMC,payment_MLAMS,payment_MLAVE,payment_MLACD,payment_MLAVS,payment_MLADC,payment_MLAAM,payment_MLAWT,payment_MLAMP,payment_MLABC,picture_width,picture_height,title_length
condition,1.000000,0.000012,0.201652,0.073602,0.055786,0.073072,0.148270,-0.066931,0.032044,0.186543,-0.011526,-0.014771,-0.013049,-0.017331,-0.010666,-0.015901,-0.012702,-0.091712,0.068287,-0.048542,-0.023592,-0.132303,0.134573
price,0.000012,1.000000,-0.001249,-0.000307,-0.000337,-0.000304,0.000320,-0.001387,-0.001146,-0.002781,-0.000385,-0.000378,-0.000380,-0.000382,-0.000400,-0.000324,-0.000370,-0.000617,-0.000389,-0.000322,0.003844,-0.004741,0.000483
automatic_relist,0.201652,-0.001249,1.000000,0.051838,0.046510,0.051363,-0.010142,-0.038168,-0.067886,0.064265,-0.017222,-0.017311,-0.017466,-0.017363,-0.018187,-0.014671,-0.016369,-0.030155,-0.020845,-0.014945,0.044045,0.074405,0.057629
initial_quantity,0.073602,-0.000307,0.051838,1.000000,0.062780,0.999948,0.029187,-0.004372,0.005498,0.023484,0.001644,0.001156,0.001385,-0.002206,0.001541,-0.003659,0.002760,-0.004020,-0.005649,-0.001812,0.017716,0.003883,0.045857
sold_quantity,0.055786,-0.000337,0.046510,0.062780,1.000000,0.057342,0.022988,0.031405,0.013708,0.017095,0.022719,0.002168,0.002393,0.000186,0.021796,-0.000538,0.023472,0.006034,-0.005486,-0.002365,0.014478,0.007152,0.034097
available_quantity,0.073072,-0.000304,0.051363,0.999948,0.057342,1.000000,0.028950,-0.004440,0.005342,0.023233,0.001518,0.001167,0.001395,-0.002180,0.001424,-0.003639,0.002629,-0.004009,-0.005594,-0.001769,0.017535,0.003804,0.045545
payment_MLATB,0.148270,0.000320,-0.010142,0.029187,0.022988,0.028950,1.000000,0.121730,0.683196,0.478243,0.069494,0.067379,0.068409,0.061612,0.070598,0.061258,0.067029,0.097399,0.077314,0.060223,0.000934,-0.030557,0.219070
payment_MLAWC,-0.066931,-0.001387,-0.038168,-0.004372,0.031405,-0.004440,0.121730,1.000000,0.111371,-0.011460,0.222951,0.220830,0.222171,0.231568,0.231540,0.212273,0.218662,0.320267,-0.022864,0.220602,-0.038422,0.009186,0.065390
payment_MLAMO,0.032044,-0.001146,-0.067886,0.005498,0.013708,0.005342,0.683196,0.111371,1.000000,0.377478,0.054543,0.053561,0.053948,0.046631,0.056429,0.046602,0.052989,0.085536,0.057259,0.044361,-0.015113,-0.043395,0.156525
payment_MLAOT,0.186543,-0.002781,0.064265,0.023484,0.017095,0.023233,0.478243,-0.011460,0.377478,1.000000,-0.038868,-0.039895,-0.039963,-0.043303,-0.040760,-0.033200,-0.036968,-0.069795,-0.010646,-0.036745,-0.003752,-0.028885,0.122178


In [30]:
top_correlated = correlation_matrix['condition'].abs().sort_values(ascending=False).drop('condition').head(10)
print(top_correlated)

automatic_relist      0.201652
payment_MLAOT         0.186543
payment_MLATB         0.148270
title_length          0.134573
picture_height        0.132303
payment_MLAWT         0.091712
initial_quantity      0.073602
available_quantity    0.073072
payment_MLAMP         0.068287
payment_MLAWC         0.066931
Name: condition, dtype: float64


In [31]:
objective_data=['automatic_relist','payment_MLAOT', 'payment_MLATB','title_length','picture_height', 'thumbnail_length']

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110078 entries, 0 to 99999
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   condition                110078 non-null  int32  
 1   listing_type_id          110078 non-null  object 
 2   price                    110078 non-null  float64
 3   accepts_mercadopago      110078 non-null  bool   
 4   automatic_relist         110078 non-null  int32  
 5   status                   110078 non-null  object 
 6   initial_quantity         110078 non-null  int64  
 7   permalink                110078 non-null  object 
 8   sold_quantity            110078 non-null  int64  
 9   available_quantity       110078 non-null  int64  
 10  shipping_free_shipping   110078 non-null  bool   
 11  shipping_mode            110078 non-null  object 
 12  geo_latitude             110078 non-null  object 
 13  geo_longitude            110078 non-null  object 
 14  payment_ML

In [38]:
object_columns = df.select_dtypes(include=['object']).columns
data_encoded = pd.get_dummies(df, columns=object_columns)


In [37]:
df.drop(columns=['permalink', 'geo_latitude', 'geo_longitude'], inplace=True)

C:\Users\memer\AppData\Local\Temp\ipykernel_16472\3101006438.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['permalink', 'geo_latitude', 'geo_longitude'], inplace=True)


In [36]:
df.nunique().sort_values(ascending=False)


permalink                  99211
geo_latitude               25906
geo_longitude              25873
price                      10283
available_quantity           468
picture_height               459
picture_width                452
initial_quantity             450
sold_quantity                317
title_length                  94
start_time_month_year         26
date_created_month_year       26
stop_time_month_year          14
last_updated_month_year       12
listing_type_id                7
shipping_mode                  4
status                         4
payment_MLADC                  2
payment_MLABC                  2
payment_MLAMP                  2
payment_MLAWT                  2
payment_MLAAM                  2
condition                      2
payment_MLAVS                  2
payment_MLACD                  2
payment_MLAVE                  2
payment_MLAMS                  2
payment_MLAMC                  2
payment_MLAMO                  2
payment_MLAWC                  2
payment_ML

In [51]:
data_encoded = pd.get_dummies(object_columns, columns=object_columns, drop_first=True)
data_encoded = pd.get_dummies(df[object_columns], drop_first=True)
data_encoded['condition'] = df['condition'].values
print(data_encoded.head())


   listing_type_id_free  listing_type_id_gold  listing_type_id_gold_premium  \
0                 False                 False                         False   
1                 False                 False                         False   
2                 False                 False                         False   
3                 False                 False                         False   
4                 False                 False                         False   

   listing_type_id_gold_pro  listing_type_id_gold_special  \
0                     False                         False   
1                     False                         False   
2                     False                         False   
3                     False                         False   
4                     False                         False   

   listing_type_id_silver  status_closed  status_not_yet_active  \
0                   False          False                  False   
1                    Tru

In [52]:
print(data_encoded.columns)


Index(['listing_type_id_free', 'listing_type_id_gold',
       'listing_type_id_gold_premium', 'listing_type_id_gold_pro',
       'listing_type_id_gold_special', 'listing_type_id_silver',
       'status_closed', 'status_not_yet_active', 'status_paused',
       'shipping_mode_me1', 'shipping_mode_me2', 'shipping_mode_not_specified',
       'last_updated_month_year_2014-12', 'last_updated_month_year_2015-01',
       'last_updated_month_year_2015-02', 'last_updated_month_year_2015-03',
       'last_updated_month_year_2015-04', 'last_updated_month_year_2015-05',
       'last_updated_month_year_2015-06', 'last_updated_month_year_2015-07',
       'last_updated_month_year_2015-08', 'last_updated_month_year_2015-09',
       'last_updated_month_year_2015-10', 'date_created_month_year_2013-07',
       'date_created_month_year_2013-08', 'date_created_month_year_2013-09',
       'date_created_month_year_2013-10', 'date_created_month_year_2014-01',
       'date_created_month_year_2014-02', 'date_cre

In [53]:
X = data_encoded.drop(columns=['condition'])  
y = data_encoded['condition'] 

In [54]:

chi2_selector = SelectKBest(chi2, k='all') 
chi2_selector.fit(X, y)

chi2_scores = chi2_selector.scores_
p_values = chi2_selector.pvalues_
chi2_results = pd.DataFrame({
    'Feature': X.columns,
    'Chi2 Score': chi2_scores,
    'P-Value': p_values
})

chi2_results = chi2_results.sort_values(by='P-Value')

print(chi2_results)


                            Feature    Chi2 Score        P-Value
0              listing_type_id_free  22732.984696   0.000000e+00
4      listing_type_id_gold_special   3291.969832   0.000000e+00
5            listing_type_id_silver   2296.659999   0.000000e+00
1              listing_type_id_gold   1033.480320  9.483520e-227
11      shipping_mode_not_specified    886.148755  1.006795e-194
..                              ...           ...            ...
31  date_created_month_year_2014-05      0.257097   6.121213e-01
23  date_created_month_year_2013-07      0.057154   8.110528e-01
61    start_time_month_year_2013-07      0.057154   8.110528e-01
39  date_created_month_year_2015-02      0.041600   8.383838e-01
77    start_time_month_year_2015-02      0.000670   9.793549e-01

[86 rows x 3 columns]


In [55]:
chi2_selector = SelectKBest(chi2, k=15)
chi2_selector.fit(X, y)

selected_features = X.columns[chi2_selector.get_support()]

print("Características seleccionadas:", selected_features)


Características seleccionadas: Index(['listing_type_id_free', 'listing_type_id_gold',
       'listing_type_id_gold_pro', 'listing_type_id_gold_special',
       'listing_type_id_silver', 'status_paused', 'shipping_mode_me1',
       'shipping_mode_me2', 'shipping_mode_not_specified',
       'last_updated_month_year_2015-08', 'last_updated_month_year_2015-09',
       'last_updated_month_year_2015-10', 'date_created_month_year_2014-10',
       'stop_time_month_year_2015-12', 'start_time_month_year_2014-12'],
      dtype='object')


In [56]:
df.head(10)


,condition,listing_type_id,price,accepts_mercadopago,automatic_relist,status,initial_quantity,sold_quantity,available_quantity,shipping_free_shipping,...,payment_MLAWT,payment_MLAMP,payment_MLABC,picture_width,picture_height,title_length,last_updated_month_year,date_created_month_year,stop_time_month_year,start_time_month_year
0,1,bronze,80.0,True,0,active,1,0,1,False,...,0,0,0,500.0,375.0,60,2015-09,2015-09,2015-11,2015-09
1,0,silver,2650.0,True,0,active,1,0,1,False,...,0,0,0,499.0,334.0,57,2015-09,2015-09,2015-11,2015-09
2,0,bronze,60.0,True,0,active,1,0,1,False,...,0,0,0,375.0,500.0,43,2015-09,2015-09,2015-11,2015-09
3,1,silver,580.0,True,0,active,1,0,1,False,...,0,0,0,441.0,423.0,52,2015-10,2015-09,2015-12,2015-09
4,0,bronze,30.0,True,0,active,1,0,1,False,...,0,0,0,375.0,500.0,25,2015-08,2015-08,2015-10,2015-08
5,1,bronze,310.0,True,0,active,1,0,1,False,...,0,0,0,250.0,250.0,41,2015-08,2015-08,2015-10,2015-08
6,0,free,180.0,True,0,active,1,0,1,False,...,0,0,0,500.0,375.0,44,2015-10,2015-10,2015-12,2015-10
7,0,free,150.0,True,0,active,1,0,1,False,...,0,0,0,500.0,375.0,52,2015-10,2015-10,2015-12,2015-10
8,1,gold_special,2352.0,True,1,paused,10,0,10,True,...,0,0,0,500.0,500.0,54,2015-10,2015-09,2015-11,2015-09
9,1,silver,120.0,True,0,active,2,1,1,False,...,0,0,0,375.0,500.0,39,2015-10,2015-10,2015-12,2015-10


In [59]:
objective_data=['automatic_relist','payment_MLAOT', 'payment_MLATB','title_length','picture_height']

column_listappend = [ 'picture_width', 'picture_height', 'title_length','listing_type_id','condition', 'shipping_mode', 'sold_quantity' , 'available_quantity'  ]

objective_columns = list(set(objective_data + column_listappend))


In [60]:
dfobjective = df[objective_columns]

In [72]:
dfobjective2 = dfobjective.copy()
dfobjective2['shipping_mode_binary'] = dfobjective2['shipping_mode'].apply(lambda x: 1 if x == 'me2' else 0)
dfobjective2.drop(columns=['shipping_mode'], inplace=True)
dfobjective2.head(10)


,picture_height,payment_MLATB,condition,sold_quantity,available_quantity,automatic_relist,title_length,picture_width,listing_type_id,payment_MLAOT,shipping_mode_binary
0,375.0,1,1,0,1,0,60,500.0,bronze,0,0
1,334.0,1,0,0,1,0,57,499.0,silver,0,1
2,500.0,1,0,0,1,0,43,375.0,bronze,1,1
3,423.0,1,1,0,1,0,52,441.0,silver,1,1
4,500.0,1,0,0,1,0,25,375.0,bronze,0,0
5,250.0,1,1,0,1,0,41,250.0,bronze,1,0
6,375.0,0,0,0,1,0,44,500.0,free,0,1
7,375.0,1,0,0,1,0,52,500.0,free,0,0
8,500.0,0,1,0,10,1,54,500.0,gold_special,1,1
9,500.0,0,1,1,1,0,39,375.0,silver,0,1


In [74]:

dfobjective2.to_csv('C:/Users/memer/OneDrive/Documentos/Eme/UNIVERSIDAD/2025-1/ETL/Tercer Cohorte/Proy85/proy85/data/dfobjective.csv',index=False)